In [2]:
#Imports
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import neighbors
from sklearn.preprocessing import LabelEncoder

In [ ]:
#Data importing
dataFrame = pd.read_csv('./dataset.csv')

# variable I will try to predict: popularity
# the dataset is taken from: https://www.kaggle.com/datasets/maharshipandya/-spotify-tracks-dataset/

# Label encoding 'track_genre'
le = LabelEncoder()
dataFrame['track_genre_encoded'] = le.fit_transform(dataFrame['track_genre'])

# Mapping of encoded labels
label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print("Label to integer mapping:", label_mapping)

# Drop the original 'track_genre' column
dataFrame = dataFrame.drop('track_genre', axis=1)

Label to integer mapping: {'acoustic': 0, 'afrobeat': 1, 'alt-rock': 2, 'alternative': 3, 'ambient': 4, 'anime': 5, 'black-metal': 6, 'bluegrass': 7, 'blues': 8, 'brazil': 9, 'breakbeat': 10, 'british': 11, 'cantopop': 12, 'chicago-house': 13, 'children': 14, 'chill': 15, 'classical': 16, 'club': 17, 'comedy': 18, 'country': 19, 'dance': 20, 'dancehall': 21, 'death-metal': 22, 'deep-house': 23, 'detroit-techno': 24, 'disco': 25, 'disney': 26, 'drum-and-bass': 27, 'dub': 28, 'dubstep': 29, 'edm': 30, 'electro': 31, 'electronic': 32, 'emo': 33, 'folk': 34, 'forro': 35, 'french': 36, 'funk': 37, 'garage': 38, 'german': 39, 'gospel': 40, 'goth': 41, 'grindcore': 42, 'groove': 43, 'grunge': 44, 'guitar': 45, 'happy': 46, 'hard-rock': 47, 'hardcore': 48, 'hardstyle': 49, 'heavy-metal': 50, 'hip-hop': 51, 'honky-tonk': 52, 'house': 53, 'idm': 54, 'indian': 55, 'indie': 56, 'indie-pop': 57, 'industrial': 58, 'iranian': 59, 'j-dance': 60, 'j-idol': 61, 'j-pop': 62, 'j-rock': 63, 'jazz': 64, 'k-pop': 65, 'kids': 66, 'latin': 67, 'latino': 68, 'malay': 69, 'mandopop': 70, 'metal': 71, 'metalcore': 72, 'minimal-techno': 73, 'mpb': 74, 'new-age': 75, 'opera': 76, 'pagode': 77, 'party': 78, 'piano': 79, 'pop': 80, 'pop-film': 81, 'power-pop': 82, 'progressive-house': 83, 'psych-rock': 84, 'punk': 85, 'punk-rock': 86, 'r-n-b': 87, 'reggae': 88, 'reggaeton': 89, 'rock': 90, 'rock-n-roll': 91, 'rockabilly': 92, 'romance': 93, 'sad': 94, 'salsa': 95, 'samba': 96, 'sertanejo': 97, 'show-tunes': 98, 'singer-songwriter': 99, 'ska': 100, 'sleep': 101, 'songwriter': 102, 'soul': 103, 'spanish': 104, 'study': 105, 'swedish': 106, 'synth-pop': 107, 'tango': 108, 'techno': 109, 'trance': 110, 'trip-hop': 111, 'turkish': 112, 'world-music': 113}

In [ ]:
inputs = ['track_genre_encoded', 'danceability', 'energy', 'key', 'loudness','mode', 'speechiness','acousticness','instrumentalness', 'liveness','valence','tempo', 'time_signature', 'explicit']
target = ['popularity']

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(dataFrame[inputs], dataFrame[target], test_size=0.2, random_state=42)

In [ ]:
for k in range(1,16):
    model = neighbors.KNeighborsRegressor(k)
    model.fit(xtrain,ytrain)
    print(f'Model Score for {k} classifiers: ', model.score(xtest, ytest))


Using KNC the best R-Squared I can get is a 0.15 so now I will move onto a neural network

In [4]:
import tensorflow as tf
import keras
#Data importing
# Data importing
dataFrame = pd.read_csv('./dataset.csv')

# Label encoding 'track_genre'
le = LabelEncoder()
dataFrame['track_genre_encoded'] = le.fit_transform(dataFrame['track_genre'])

# Drop the columns
dataFrame = dataFrame.drop(columns=['Unnamed: 0', 'track_id', 'artists', 'album_name', 'track_name', 'track_genre'])

# Split into features and target
X = dataFrame.drop(columns=['popularity'])
y = dataFrame['popularity']

# Convert to tensor
X_tensor = tf.convert_to_tensor(X.values, dtype=tf.float32)
y_tensor = tf.convert_to_tensor(y.values, dtype=tf.float32)

# Calculate the index at which to split the dataset
train_size = int(0.8 * len(X))

# Manually slice the tensors for train/test sets
x_train_tensor = X_tensor[:train_size]
y_train_tensor = y_tensor[:train_size]
x_test_tensor = X_tensor[train_size:]
y_test_tensor = y_tensor[train_size:]


In [5]:
@keras.saving.register_keras_serializable()
def mse_loss(y_pred, y_true):
    return tf.reduce_mean(tf.square(y_pred - y_true))

@keras.saving.register_keras_serializable()
def rmse(y_true, y_pred):
    return tf.sqrt(mse_loss(y_pred, y_true))

@keras.saving.register_keras_serializable()
def r_squared(y_true, y_pred):
    residual = tf.reduce_sum(tf.square(tf.subtract(y_true, y_pred)))
    total = tf.reduce_sum(tf.square(tf.subtract(y_true, tf.reduce_mean(y_true))))
    r2 = tf.subtract(1.0, tf.divide(residual, total))
    return r2

# Used to trace history of RMSE as the neural network gets trained
class RMSEHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        if logs is None:
            logs = {}
        self.modelRMSE = []
        self.validationRMSE = []

    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
        self.modelRMSE.append(logs.get('rmse'))
        self.validationRMSE.append(logs.get('val_rmse'))

In [ ]:
from matplotlib import pyplot as plt
from tensorflow.python.keras.callbacks import EarlyStopping

input_shape = (x_train_tensor.shape[1],)

rmse_history = RMSEHistory()

baseNum = 4

# Define your original model with regularization
def build_model(input_shape, regularization_factor=0.01):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(baseNum*2, activation='relu', input_shape=input_shape),
        tf.keras.layers.Dense(baseNum*4, activation='elu'),
        tf.keras.layers.Dense(baseNum*4, activation='elu'),
        tf.keras.layers.Dense(baseNum*2, activation='linear'),
        # tf.keras.layers.Dense(baseNum, activation='relu', kernel_regularizer=regularizers.l2(regularization_factor)),
        tf.keras.layers.Dense(1)
    ])
    return model

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)

with tf.device('/GPU:0'):
    #regular model training
    model = build_model(input_shape)
    model.compile(optimizer='adam', loss=mse_loss, metrics=[r_squared, rmse])
    model.fit(x_train_tensor, y_train_tensor, epochs=500, batch_size=100, validation_split=0.2, callbacks=[early_stopping,rmse_history])
    # if no improvement has been made in 100 generations (epochs) stop the model
    if early_stopping.stopped_epoch > 0:
        print(f"Early stopping occurred at epoch {early_stopping.stopped_epoch}")
        print(f"Restoring model weights from the end of the best epoch.")
    else:
        print("Early stopping did not occur.")

    test_metrics = model.evaluate(x_test_tensor, y_test_tensor)
    test_loss, test_accuracy, test_rmse = test_metrics[0], test_metrics[1], test_metrics[2]
    print(f"Test Loss (Accuracy): {test_loss}")
    print(f"Test Accuracy: {test_accuracy*100}")
    print(f"Test RMSE: {test_rmse}")
    # Plotting RMSE values
    plt.figure(figsize=(10, 5))
    plt.plot(rmse_history.modelRMSE, label='Train RMSE')
    plt.plot(rmse_history.validationRMSE, label='Validation RMSE')
    plt.xlabel('Generation')
    plt.ylabel('RMSE')
    plt.title('RMSE During Training')
    plt.legend()
    plt.show()

Epoch 1/500
1460/1460 [==============================] - 2s 1ms/step - loss: 3238531.5000 - r_squared: -5857.9282 - rmse: 263.3191 - val_loss: 968.2595 - val_r_squared: -54.1650 - val_rmse: 28.1987
Epoch 2/500
1460/1460 [==============================] - 1s 933us/step - loss: 1279.4233 - r_squared: -1.5719 - rmse: 32.1951 - val_loss: 1808.6515 - val_r_squared: -265.1624 - val_rmse: 40.1310
Epoch 3/500
1460/1460 [==============================] - 1s 899us/step - loss: 4132.0972 - r_squared: -7.4352 - rmse: 44.9316 - val_loss: 882366.1250 - val_r_squared: -115700.5078 - val_rmse: 908.4353
Epoch 4/500
1460/1460 [==============================] - 1s 902us/step - loss: 97545.0000 - r_squared: -196.3405 - rmse: 125.3006 - val_loss: 782.9655 - val_r_squared: -92.6610 - val_rmse: 26.3345
Epoch 5/500
1460/1460 [==============================] - 1s 894us/step - loss: 113872.4141 - r_squared: -226.2620 - rmse: 100.4903 - val_loss: 1684.5358 - val_r_squared: -128.1632 - val_rmse: 38.0736
Epoch 6/5